----

In [7]:
%pip install datasets --quiet
%pip install transformers --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
import datasets 
from transformers import BertTokenizer

# squad_ds_list = [ ds for ds in datasets.list_datasets() if 'squad' in ds.lower() ]

In [14]:
## Example of loading data with streaming = True
dataset = datasets.load_dataset('squad', streaming=True)

print(list(dataset.keys()))
print(dataset['train'].description)
print(f"The Length of Data is : {dataset['train'].dataset_size: ,} records")
print(dataset['train'].features)

## define the transformation to dataset['train'] here
dataset['train'] = dataset['train'].map(
    lambda x: {
        'id': x['id'],
        'answers': { 
            **x['answers'], 
            **{
                'answer_end': [ x['answers']['answer_start'][0] + \
                               len(x['answers']['text'][0]) ]
            }
        },
        'context': x['context'],
        'question': x['question'],
        'title': x['title']
    }
)

## Since streaming true then beaware of lazy exec
print("Printing Samples from Dataset")
for i, sample in enumerate(dataset['train']):
    print(sample['answers'], '\n')
    if i > 4: break

['train', 'validation']
Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

The Length of Data is :  89,819,092 records
{'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}
Printing Samples from Dataset
{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515], 'answer_end': [541]} 

{'text': ['a copper statue of Christ'], 'answer_start': [188], 'answer_end': [213]} 

{'text': ['the Main Building'], 'answer_start': [279], 'answer_end': [296]} 

{'text': ['a Marian place of prayer and re

In [17]:
## Example of same as above when streaming is False
dataset = datasets.load_dataset('squad', streaming=False)

print(list(dataset.keys()))
print(dataset['train'].description)
print(f"The Length of Data is : {dataset['train'].dataset_size: ,} records")
print(dataset['train'].features)

## define the transformation to dataset['train'] here
dataset['train'] = dataset['train'].map(
    lambda x: {
        'answers': { 
            **x['answers'], 
            **{
                'answer_end': [ x['answers']['answer_start'][0] + \
                               len(x['answers']['text'][0]) ]
            }
        }
    }
)

print(f"Printing Features from train dataset Dataset: {dataset['train'].features}")

# rename column
dataset['train'] = dataset['train'].rename_column('title', 'topic') 

# filter 
dataset['train'] = dataset['train'].filter(
    lambda x: x['topic'] == 'University_of_Notre_Dame' 
)

Found cached dataset squad (/Users/mustafawaheed/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/mustafawaheed/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-cf966adab71fdca4.arrow
Loading cached processed dataset at /Users/mustafawaheed/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-040c5c3fef3797ac.arrow


['train', 'validation']
Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

The Length of Data is :  89,819,092 records
{'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}
Printing Features from train dataset Dataset: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': {'answer_end': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'answer_start':

In [20]:
len(dataset['train'])

269

In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

dataset['train'] = dataset['train'].map(
    lambda x: tokenizer(
        x['question'], 
        x['context'],
        max_length = 512,
        padding = 'max_length',
        truncation = True
    ), batched = True, batch_size=32
)


Map:   0%|          | 0/269 [00:00<?, ? examples/s]